In [4]:
from difflib import get_close_matches

def phonetic_encode(word):

    replacements = {
        'அ': 'a', 'ஆ': 'aa', 'இ': 'i', 'ஈ': 'ii', 'உ': 'u', 'ஊ': 'uu',
        'எ': 'e', 'ஏ': 'ee', 'ஐ': 'ai', 'ஒ': 'o', 'ஓ': 'oo', 'ஔ': 'au',
        'க': 'k', 'ச': 's', 'ட': 't', 'த': 'th', 'ப': 'p', 'ற': 'r',
        'ன': 'n', 'ந': 'n', 'ம': 'm', 'ய': 'y', 'ர': 'r', 'ல': 'l',
        'வ': 'v', 'ழ': 'zh', 'ள': 'l', 'ஜ': 'j', 'ஷ': 'sh', 'ஸ': 's', 'ஹ': 'h'
    }

    encoded_word = ""
    for char in word:
        encoded_word += replacements.get(char, char)

    return encoded_word

def spell_checker(input_word, tamil_dictionary):

    input_encoded = phonetic_encode(input_word)
    dictionary_encoded = {word: phonetic_encode(word) for word in tamil_dictionary}

    closest_matches = get_close_matches(input_encoded, dictionary_encoded.values(), n=3, cutoff=0.6)
    
    suggestions = [
        word for word, encoded in dictionary_encoded.items() if encoded in closest_matches
    ]

    return suggestions

if __name__ == "__main__":
    tamil_dictionary = [
        "காத்து", "காதல்", "காலம்", "மலர்", "மழை", "வானம்", "நதி"
    ]

    input_word = "காலம"

    suggestions = spell_checker(input_word, tamil_dictionary)

    if suggestions:
        print(f"Suggestions for '{input_word}': {', '.join(suggestions)}")
    else:
        print(f"No suggestions found for '{input_word}'.")


Suggestions for 'காலம': காதல், காலம்
